In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model_save_dir = "/content/drive/My Drive/my_keras_models/eurosat_models/"

# Create the directory if it doesn't exist
# exist_ok=True prevents error if directory already exists
os.makedirs(model_save_dir, exist_ok=True)

model_filepath = os.path.join(model_save_dir, "eurosat_RGB_img_classification")

print(f"Model filepath for saving new models: {model_filepath}")

In [ ]:
!wget https://zenodo.org/records/7711810/files/EuroSAT_RGB.zip?download=1
!mv "EuroSAT_RGB.zip?download=1" "EuroSAT_RGB.zip"
!unzip "EuroSAT_RGB.zip"

In [ ]:
DATA_DIR = "EuroSAT_RGB"
CATEGORIES = os.listdir(DATA_DIR)

print("All categories:")
for cat in CATEGORIES:
    print("\t - " + cat)

In [ ]:
training_data = []

# Redefining category order to fix wrong data classification
CATEGORIES = [
    'AnnualCrop',
    'Forest',
    'HerbaceousVegetation',
    'Highway',
    'Industrial',
    'Pasture',
    'PermanentCrop',
    'Residential',
    'River',
    'SeaLake'
]

def preprocess_im_array(img_array):
    img_array = img_array.astype(np.float32)
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    stddev = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    normalized_image = (img_array - mean) / stddev
    return normalized_image

print("All categories re-ordered:")
for cat in CATEGORIES:
    print("\t - " + cat)

def create():
    for category in CATEGORIES:
        path = os.path.join(DATA_DIR, category)
        class_num = CATEGORIES.index(category)
        for image in os.listdir(path):
            try:
                image_path = os.path.join(path,image)
                resized_image_array = cv2.resize(cv2.imread(image_path), (64, 64))
                _img_array = np.array(resized_image_array)
                final_img_array = preprocess_im_array(_img_array)
                print("Pre-processed image")
                training_data.append([final_img_array, class_num])
            except FileNotFoundError:
                print(f"File not found: {os.path.join(path, image)}")
            except cv2.error as e:
                print(f"OpenCV error processing {os.path.join(path, image)}: {e}")
            except Exception as e:
                print(f"An unexpected error occured processing {os.path.join(path,image)}: {e}")

create()

print("Length of training data: " + str(len(training_data)))

In [ ]:
# this step prepares the data from a list of (image, label) pairs into the
# format that TensorFlow/Keras can efficiently process for training my model.

Feats = []
Labels = []

for feats, label in training_data:
    Feats.append(feats)
    Labels.append(label)

Feats = np.array(Feats)
Labels = np.array(Labels)

print("Features and labels converted to NumPy arrays (for TensorFlow/Keras)")

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

X_train, X_test, Y_train, Y_test = train_test_split(Feats, Labels, test_size=0.2, random_state=42)
print("Splitting data success")

X_train = X_train / 255.0
X_test = X_test / 255.0
print("Normalizing pixels success")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(CATEGORIES), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("Built model")

In [ ]:
history = model.fit(X_train, Y_train, epochs=5, validation_data=(X_test, Y_test))
print("Model trained!")

In [ ]:
model.export(model_filepath)
print(f"Model saved to: {model_filepath}")

In [ ]:
!pip install --upgrade tf2onnx

In [ ]:
import tf2onnx
import numpy as np
import os

# Get the tf2onnx installation directory
tf2onnx_dir = os.path.dirname(tf2onnx.__file__)
tfonnx_path = os.path.join(tf2onnx_dir, "tfonnx.py")

print(f"Attempting to patch tf2onnx file at: {tfonnx_path}")

try:
    with open(tfonnx_path, 'r') as f:
        content = f.read()

    # Replace np.cast with np.asarray in the specific function
    updated_content = content.replace("np.cast", "np.asarray")

    with open(tfonnx_path, 'w') as f:
        f.write(updated_content)

    print("Successfully attempted to patch tf2onnx.")

except Exception as e:
    print(f"An error occurred while trying to patch tf2onnx: {e}")

# Now, try the ONNX conversion again
!python -m tf2onnx.convert --saved-model /content/drive/MyDrive/my_keras_models/eurosat_models/eurosat_RGB_img_classification --output EuroSat_RGB_prep.onnx --opset 15